In [45]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from pyspark.sql import SparkSession
# Database connection parameters
db_host = 'database-1.cxoaueie6ybk.us-east-2.rds.amazonaws.com'
db_port = 5432  # Default PostgreSQL port is 5432
db_name = 'postgres'
db_user = 'root'
db_password = 'irODJyh3LQpE0V3OcE3o'

# Create an engine to connect to the database
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Establish a connection to the database
conn = psycopg2.connect(
    host=db_host,
    port=db_port,
    dbname=db_name,
    user=db_user,
    password=db_password
)
# SQL query to fetch data
query = 'SELECT * FROM darwin'

# Load data into a DataFrame
df = pd.read_sql(query, con = conn)

df

/tmp/ipykernel_166/1125145934.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con = conn)


,route_id,unique_id,service_start_date,update_origin,train_platform,working_time_pass,working_time_arrival,working_time_departure,planned_time_arrival,planned_time_departure,...,platform,train_length,estimated_time,source,actual_time,actual_time_class,is_delayed_arrival,is_delayed_departure,source_instance,estimated_time_minutes
0,202407038081744,P81744,2024-07-03,TD,CCRT,None,23:17:00,23:17:30,23:17:00,23:17:00,...,2,NaN,None,None,None,None,False,False,None,None
1,202407038097422,P97422,2024-07-03,TD,ALBNYPK,None,23:14:00,23:14:30,23:14:00,23:14:00,...,2,8.0,None,None,None,None,False,False,None,None
2,202407038097422,P97422,2024-07-03,TD,BEXLEY,None,23:16:30,23:17:30,23:17:00,23:17:00,...,2,8.0,None,None,None,None,False,False,None,None
3,202407037143658,G43658,2024-07-03,TD,SHAL341,None,23:29:00,23:32:00,None,None,...,None,NaN,None,None,None,None,False,False,None,None
4,202407037143658,G43658,2024-07-03,TD,SHALFD,None,23:34:00,23:35:30,None,None,...,None,NaN,None,None,None,None,False,False,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261108,202407087132571,G32571,2024-07-08,TD,BNSLY,None,15:20:30,15:21:30,15:21:00,15:21:00,...,1,2.0,None,None,None,None,False,False,None,None
261109,202407087132571,G32571,2024-07-08,TD,WOMBWEL,None,15:26:30,15:27:00,15:27:00,15:27:00,...,1,2.0,None,None,None,None,False,False,None,None
261110,202407087132571,G32571,2024-07-08,TD,ELSC,None,15:30:00,15:31:00,15:30:00,15:31:00,...,1,2.0,None,None,None,None,False,False,None,None
261111,202407087132571,G32571,2024-07-08,TD,CHPLTWN,None,15:36:00,15:36:30,15:36:00,15:36:00,...,1,2.0,None,None,None,None,False,False,None,None


In [36]:
from pyspark.sql import SparkSession
# Initialize Spark session
spark = SparkSession.builder \
    .appName("Pandas to PySpark") \
    .getOrCreate()

#not working current as AttributeError: 'datetime.time' object has no attribute 'timetuple'
spark_df = spark.createDataFrame(df)
spark_df.printScheme()

AttributeError: 'datetime.time' object has no attribute 'timetuple'

In [46]:
#getting an overview of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261113 entries, 0 to 261112
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   route_id                261113 non-null  object 
 1   unique_id               261113 non-null  object 
 2   service_start_date      261113 non-null  object 
 3   update_origin           254135 non-null  object 
 4   train_platform          261113 non-null  object 
 5   working_time_pass       0 non-null       object 
 6   working_time_arrival    261113 non-null  object 
 7   working_time_departure  261113 non-null  object 
 8   planned_time_arrival    236790 non-null  object 
 9   planned_time_departure  235823 non-null  object 
 10  actual_arrival_time     218666 non-null  object 
 11  actual_departure_time   242953 non-null  object 
 12  platform                245358 non-null  object 
 13  train_length            91502 non-null   float64
 14  estimated_time      

In [47]:
#dropping null columns and showing new dataframe overview
df = df.drop(['working_time_pass','estimated_time','source','actual_time','actual_time_class','source_instance','estimated_time_minutes'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261113 entries, 0 to 261112
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   route_id                261113 non-null  object 
 1   unique_id               261113 non-null  object 
 2   service_start_date      261113 non-null  object 
 3   update_origin           254135 non-null  object 
 4   train_platform          261113 non-null  object 
 5   working_time_arrival    261113 non-null  object 
 6   working_time_departure  261113 non-null  object 
 7   planned_time_arrival    236790 non-null  object 
 8   planned_time_departure  235823 non-null  object 
 9   actual_arrival_time     218666 non-null  object 
 10  actual_departure_time   242953 non-null  object 
 11  platform                245358 non-null  object 
 12  train_length            91502 non-null   float64
 13  is_delayed_arrival      261113 non-null  bool   
 14  is_delayed_departure

In [48]:
#checking duplicates
df.duplicated().sum()

9407

In [49]:
#dropping duplicates
df = df.drop_duplicates()
df.duplicated().sum()

0

In [50]:
#taking in rail reference csv to extract information about the stations routes are interacting with
df2 = pd.read_csv("./Data/RailReferences.csv")
df2.head()

,AtcoCode,TiplocCode,CrsCode,StationName,StationNameLang,GridType,Easting,Northing,CreationDateTime,ModificationDateTime,RevisionNumber,Modification
0,9100PENZNCE,PENZNCE,PNZ,Penzance Rail Station,NaN,U,147588,30599,2003-11-04T00:00:00,2011-09-30T14:47:28,2,rev
1,9100STIVES,STIVES,SIV,St Ives (Cornwall) Rail Station,NaN,U,151947,40127,2005-04-04T00:00:00,2011-09-30T14:47:28,4,rev
2,9100CARBISB,CARBISB,CBB,Carbis Bay Rail Station,NaN,U,152930,38745,2003-11-04T00:00:00,2011-09-30T14:47:28,5,rev
3,9100STERTH,STERTH,SER,St Erth Rail Station,NaN,U,154150,35730,2003-11-04T00:00:00,2007-02-13T17:06:21,3,rev
4,9100LELANTS,LELANTS,LTS,Lelant Saltings Rail Station,NaN,U,154430,36640,2003-11-04T00:00:00,2011-09-30T14:47:28,2,rev


In [51]:
#dropping information about the rail stations that we do not need
df2 = df2.drop(["StationNameLang","GridType","AtcoCode","Modification","CreationDateTime","ModificationDateTime","RevisionNumber","CrsCode"],axis=1)
df2

,TiplocCode,StationName,Easting,Northing
0,PENZNCE,Penzance Rail Station,147588,30599
1,STIVES,St Ives (Cornwall) Rail Station,151947,40127
2,CARBISB,Carbis Bay Rail Station,152930,38745
3,STERTH,St Erth Rail Station,154150,35730
4,LELANTS,Lelant Saltings Rail Station,154430,36640
...,...,...,...,...
2623,SMRLYTN,Somerleyton Rail Station,647930,296530
2624,OULTNBS,Oulton Broad South Rail Station,651920,292205
2625,YARMTH,Great Yarmouth Rail Station,652000,308100
2626,OULTNBN,Oulton Broad North Rail Station,652420,293140


In [52]:
#merging together the information about the train platform for each route
merged_df = pd.merge(df, df2, left_on='train_platform', right_on='TiplocCode', how='left')


In [53]:
#showing the new dataframe with station information
merged_df.head()

,route_id,unique_id,service_start_date,update_origin,train_platform,working_time_arrival,working_time_departure,planned_time_arrival,planned_time_departure,actual_arrival_time,actual_departure_time,platform,train_length,is_delayed_arrival,is_delayed_departure,TiplocCode,StationName,Easting,Northing
0,202407038081744,P81744,2024-07-03,TD,CCRT,23:17:00,23:17:30,23:17:00,23:17:00,None,23:17:00,2,NaN,False,False,CCRT,Cathcart Rail Station,258460.0,660580.0
1,202407038097422,P97422,2024-07-03,TD,ALBNYPK,23:14:00,23:14:30,23:14:00,23:14:00,None,None,2,8.0,False,False,ALBNYPK,Albany Park Rail Station,547855.0,172860.0
2,202407038097422,P97422,2024-07-03,TD,BEXLEY,23:16:30,23:17:30,23:17:00,23:17:00,23:18:00,23:19:00,2,8.0,False,False,BEXLEY,Bexley Rail Station,549380.0,173435.0
3,202407037143658,G43658,2024-07-03,TD,SHAL341,23:29:00,23:32:00,None,None,23:30:00,23:31:00,None,NaN,False,False,NaN,NaN,NaN,NaN
4,202407037143658,G43658,2024-07-03,TD,SHALFD,23:34:00,23:35:30,None,None,23:33:00,23:34:00,None,NaN,False,False,SHALFD,Shalford (Surrey) Rail Station,500200.0,147100.0


In [54]:
#setting the working df to the merged df and dropping TiplocCode as it is no longer needed
df = merged_df.drop(["TiplocCode"],axis = 1)

In [55]:
df.head()

,route_id,unique_id,service_start_date,update_origin,train_platform,working_time_arrival,working_time_departure,planned_time_arrival,planned_time_departure,actual_arrival_time,actual_departure_time,platform,train_length,is_delayed_arrival,is_delayed_departure,StationName,Easting,Northing
0,202407038081744,P81744,2024-07-03,TD,CCRT,23:17:00,23:17:30,23:17:00,23:17:00,None,23:17:00,2,NaN,False,False,Cathcart Rail Station,258460.0,660580.0
1,202407038097422,P97422,2024-07-03,TD,ALBNYPK,23:14:00,23:14:30,23:14:00,23:14:00,None,None,2,8.0,False,False,Albany Park Rail Station,547855.0,172860.0
2,202407038097422,P97422,2024-07-03,TD,BEXLEY,23:16:30,23:17:30,23:17:00,23:17:00,23:18:00,23:19:00,2,8.0,False,False,Bexley Rail Station,549380.0,173435.0
3,202407037143658,G43658,2024-07-03,TD,SHAL341,23:29:00,23:32:00,None,None,23:30:00,23:31:00,None,NaN,False,False,NaN,NaN,NaN
4,202407037143658,G43658,2024-07-03,TD,SHALFD,23:34:00,23:35:30,None,None,23:33:00,23:34:00,None,NaN,False,False,Shalford (Surrey) Rail Station,500200.0,147100.0


In [56]:
#looking at null values accross the dataframe
df.isna().sum()

route_id                       0
unique_id                      0
service_start_date             0
update_origin               6024
train_platform                 0
working_time_arrival           0
working_time_departure         0
planned_time_arrival       24163
planned_time_departure     24968
actual_arrival_time        42277
actual_departure_time      17944
platform                   15489
train_length              162675
is_delayed_arrival             0
is_delayed_departure           0
StationName                19326
Easting                    19326
Northing                   19326
dtype: int64

In [57]:
#dropping the nulls in Easting and Northing before converting them to Latitude and Longitude
df = df.dropna(subset=['Easting', 'Northing'])

In [58]:
#installing a necessary package to convert northing and easting to latitude and longitude
!pip install bng_latlon

In [59]:
# Function to convert to Latitude and Longitude
# !pip install bng_latlon
from bng_latlon import OSGB36toWGS84
def latlong(df):
    lat_long = []
    for i in range(len(df)):
        lat_long.append( OSGB36toWGS84(df[i][0],df[i][1]) )
    return lat_long
df['Easting'] = df['Easting'].astype('int64')
df['Northing'] = df['Northing'].astype('int64')
df1 = df [['Easting','Northing']]
values = (df1.values)
lat_long = latlong(values)
#Writing to dataframe
lat = []
long = []
for i in range(len(lat_long)):
    lat.append(lat_long[i][0])
    long.append(lat_long[i][1])
    
df['Latitude'] = lat
df['Longitude'] = long

In [25]:
#looking at the data after adding latitude and longitude columns
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227020 entries, 0 to 246085
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   route_id                227020 non-null  object        
 1   unique_id               227020 non-null  object        
 2   service_start_date      227020 non-null  datetime64[ns]
 3   update_origin           221338 non-null  object        
 4   train_platform          227020 non-null  object        
 5   working_time_arrival    227020 non-null  object        
 6   working_time_departure  227020 non-null  object        
 7   planned_time_arrival    213261 non-null  object        
 8   planned_time_departure  212448 non-null  object        
 9   actual_arrival_time     187772 non-null  object        
 10  actual_departure_time   210249 non-null  object        
 11  platform                222583 non-null  object        
 12  train_length            82835 

In [60]:
#creating a new dataframe dropping all null values from the data
na_df = df.dropna()

In [61]:
#getting an overview of the amount of rows dropped and if we should continue with the null dropped dataset
na_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65019 entries, 2 to 251705
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   route_id                65019 non-null  object 
 1   unique_id               65019 non-null  object 
 2   service_start_date      65019 non-null  object 
 3   update_origin           65019 non-null  object 
 4   train_platform          65019 non-null  object 
 5   working_time_arrival    65019 non-null  object 
 6   working_time_departure  65019 non-null  object 
 7   planned_time_arrival    65019 non-null  object 
 8   planned_time_departure  65019 non-null  object 
 9   actual_arrival_time     65019 non-null  object 
 10  actual_departure_time   65019 non-null  object 
 11  platform                65019 non-null  object 
 12  train_length            65019 non-null  float64
 13  is_delayed_arrival      65019 non-null  bool   
 14  is_delayed_departure    65019 non-nul

In [62]:
#after considering the data dropped we believe the amount of data still remaining is sufficent to run our EDA
df = na_df

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65019 entries, 2 to 251705
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   route_id                65019 non-null  object 
 1   unique_id               65019 non-null  object 
 2   service_start_date      65019 non-null  object 
 3   update_origin           65019 non-null  object 
 4   train_platform          65019 non-null  object 
 5   working_time_arrival    65019 non-null  object 
 6   working_time_departure  65019 non-null  object 
 7   planned_time_arrival    65019 non-null  object 
 8   planned_time_departure  65019 non-null  object 
 9   actual_arrival_time     65019 non-null  object 
 10  actual_departure_time   65019 non-null  object 
 11  platform                65019 non-null  object 
 12  train_length            65019 non-null  float64
 13  is_delayed_arrival      65019 non-null  bool   
 14  is_delayed_departure    65019 non-nul

In [64]:
df.describe()

,train_length,Easting,Northing,Latitude,Longitude
count,65019.000000,65019.00000,65019.000000,65019.000000,65019.000000
mean,7.083637,523287.23184,199344.538627,51.676758,-0.228325
std,2.792233,54577.65309,93431.653304,0.844081,0.788920
min,2.000000,154150.00000,35730.000000,50.170489,-5.444252
25%,4.000000,519433.00000,157613.000000,51.296488,-0.266320
50%,8.000000,531714.00000,175500.000000,51.463498,-0.103764
75%,8.000000,546090.00000,196666.000000,51.653068,0.101690
max,12.000000,652420.00000,709600.000000,56.273781,1.715711


In [65]:
from datetime import time
#changing the columns with times to datetime objects
df['service_start_date'] = pd.to_datetime(df['service_start_date'])

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65019 entries, 2 to 251705
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   route_id                65019 non-null  object        
 1   unique_id               65019 non-null  object        
 2   service_start_date      65019 non-null  datetime64[ns]
 3   update_origin           65019 non-null  object        
 4   train_platform          65019 non-null  object        
 5   working_time_arrival    65019 non-null  object        
 6   working_time_departure  65019 non-null  object        
 7   planned_time_arrival    65019 non-null  object        
 8   planned_time_departure  65019 non-null  object        
 9   actual_arrival_time     65019 non-null  object        
 10  actual_departure_time   65019 non-null  object        
 11  platform                65019 non-null  object        
 12  train_length            65019 non-null  float

In [66]:
#creating a calculated column to show the difference to time between the planned and actual arrival
from datetime import time, timedelta
df['arrival_vs_planned_arrival'] = df.apply(lambda row: timedelta(hours=row['actual_arrival_time'].hour - row['planned_time_arrival'].hour,
                                                       minutes=row['actual_arrival_time'].minute - row['planned_time_arrival'].minute,
                                                       seconds=row['actual_arrival_time'].second - row['planned_time_arrival'].second), axis=1)


df['arrival_vs_planned_arrival']

2        0 days 00:01:00
25       0 days 00:00:00
37       0 days 00:01:00
38       0 days 00:01:00
44       0 days 00:00:00
               ...      
251701   0 days 00:08:00
251702   0 days 00:07:00
251703   0 days 00:08:00
251704   0 days 00:08:00
251705   0 days 00:07:00
Name: arrival_vs_planned_arrival, Length: 65019, dtype: timedelta64[ns]

In [67]:
#creating a calculated column to show the difference to time between the planned and actual departure
#creating a calculated column to show the difference to time between the planned and actual arrival
from datetime import time, timedelta
df['arrival_vs_planned_departure'] = df.apply(lambda row: timedelta(hours=row['actual_departure_time'].hour - row['planned_time_departure'].hour,
                                                       minutes=row['actual_departure_time'].minute - row['planned_time_departure'].minute,
                                                       seconds=row['actual_departure_time'].second - row['planned_time_departure'].second), axis=1)


df['arrival_vs_planned_departure']

2        0 days 00:02:00
25       0 days 00:00:00
37       0 days 00:01:00
38       0 days 00:01:00
44       0 days 00:00:00
               ...      
251701   0 days 00:08:00
251702   0 days 00:08:00
251703   0 days 00:08:00
251704   0 days 00:08:00
251705   0 days 00:07:00
Name: arrival_vs_planned_departure, Length: 65019, dtype: timedelta64[ns]

In [68]:
#looking at data before exporting to csv file
df.head()

,route_id,unique_id,service_start_date,update_origin,train_platform,working_time_arrival,working_time_departure,planned_time_arrival,planned_time_departure,actual_arrival_time,...,train_length,is_delayed_arrival,is_delayed_departure,StationName,Easting,Northing,Latitude,Longitude,arrival_vs_planned_arrival,arrival_vs_planned_departure
2,202407038097422,P97422,2024-07-03,TD,BEXLEY,23:16:30,23:17:30,23:17:00,23:17:00,23:18:00,...,8.0,False,False,Bexley Rail Station,549380,173435,51.440218,0.147903,0 days 00:01:00,0 days 00:02:00
25,202407037139898,G39898,2024-07-03,CIS,BOXHAWH,23:17:30,23:18:00,23:18:00,23:18:00,23:18:00,...,10.0,False,False,Box Hill & Westhumble Rail Station,516744,151865,51.254010,-0.328490,0 days 00:00:00,0 days 00:00:00
37,202407037154196,G54196,2024-07-03,CIS,HYWRDSH,23:16:00,23:17:00,23:16:00,23:17:00,23:17:00,...,12.0,False,False,Haywards Heath Rail Station,533045,124630,51.005680,-0.105078,0 days 00:01:00,0 days 00:01:00
38,202407037154196,G54196,2024-07-03,CIS,THBDGS,23:27:30,23:29:00,23:28:00,23:29:00,23:29:00,...,12.0,False,False,Three Bridges Rail Station,528800,136900,51.116922,-0.161184,0 days 00:01:00,0 days 00:01:00
44,202407037150851,G50851,2024-07-03,TD,HRPNDN,23:22:00,23:22:30,23:22:00,23:22:00,23:22:00,...,12.0,False,False,Harpenden Rail Station,513735,214175,51.814643,-0.351481,0 days 00:00:00,0 days 00:00:00


In [69]:
#converting the dataframe into a csv file to be exported into other programs like PowerBI
df.to_csv('./Data/cleanRailData.csv')

In [71]:
#getting the average delay for each station
avg_time_difference = df.groupby('StationName')['arrival_vs_planned_arrival'].apply(lambda x: x.mean())

avg_time_difference

StationName
Abbey Wood (London) Rail Station    0 days 00:03:10.838709677
Accrington Rail Station                       0 days 00:03:36
Acle Rail Station                   0 days 00:00:17.142857142
Adisham Rail Station                0 days 00:05:17.142857142
Adlington (Cheshire) Rail Station             0 days 00:02:00
                                               ...           
Wylam Rail Station                            0 days 00:00:48
Wymondham Rail Station              0 days 00:01:26.666666666
Yalding Rail Station                0 days 00:02:45.714285714
Yarm Rail Station                   0 days 00:06:44.210526315
York Rail Station                   0 days 00:09:58.536585365
Name: arrival_vs_planned_arrival, Length: 1210, dtype: timedelta64[ns]

In [72]:
#importing plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns